In [1]:
import os
os.environ["WANDB_API_KEY"] = "dcf9600e0485401cbb0ddbb0f7be1c70f96b32ef"
os.environ["WANDB_MODE"] = "disabled"
import argparse
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import ipdb
import wandb
from sigma_layer import SigmaLinear, SigmaConv, SigmaView
from utils import get_dataset, gradient_centralization, normalize_along_axis, get_activation_function, compute_SCL_loss
import datetime

In [2]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
# Training scheme group
method_parser = parser.add_argument_group("Method")
method_parser.add_argument('--method', type=str, default='SIGMA', choices=['SIGMA', 'BP', 'FA'])
method_parser.add_argument('--architecture', type=str, default='lenet', choices=['lenet', 'vgg'])
method_parser.add_argument('--actfunc', type=str, default='elu', choices=['tanh', 'elu', 'relu'])
method_parser.add_argument('--conv_dim', type=int, default=32, choices=[32, 64])
# Dataset group
dataset_parser = parser.add_argument_group('Dataset')
dataset_parser.add_argument('--dataset', type=str, default='CIFAR10', choices=['MNIST', 'CIFAR10'])
dataset_parser.add_argument('--batchsize', type=int, default=64)
dataset_parser.add_argument('--splitratio', type=float, default=0.2)
# Training group # LR, optimizer, weight_decay, momentum
training_parser = parser.add_argument_group('Training')
training_parser.add_argument('--epochs', type=int, default=100)
training_parser.add_argument('--lr', type=float, default=1)
training_parser.add_argument('--optimizer', type=str, default='SGD', choices=['RMSprop', 'Adam', 'SGD'])
# Seed group
seed_parser = parser.add_argument_group('Seed')
seed_parser.add_argument('--seed', type=int, default=2023)
args, _ = parser.parse_known_args()

# Set run_name
run_name = f"{args.dataset}_{args.method}_conv{args.conv_dim}-act{args.actfunc}_{args.optimizer}_{args.lr}_{args.seed}"
time_stamp = datetime.datetime.now().strftime("%m-%d_%H-%M-%S")

# Set wandb
wandb.init(
    project="opt-sigma",
    name=run_name,
    # track hyperparameters and run metadata
    config={
    "algorithm": args.method,
    "architecture": "SimpleCNN",
    "dataset": args.dataset,
    "epochs": args.epochs,
    "lr": args.lr,
    "optimizer": args.optimizer,
    "seed": args.seed,
    "conv_dim": args.conv_dim,
    "actfunc": args.actfunc,
    }
)

print(f"Run name: {run_name}")

# Set seed
torch.manual_seed(args.seed), np.random.seed(args.seed)

# Set device
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")

# Get dataset
train_loader, val_loader, test_loader = get_dataset(args)

class SigmaModel_SimpleCNN(nn.Module):
    def __init__(self, args):
        super(SigmaModel_SimpleCNN, self).__init__()
        if args.dataset == "CIFAR10":
            self.LN1 = torch.nn.LayerNorm((3,32,32), elementwise_affine=False) 
            self.conv1 = SigmaConv(args, 3, 32, 5, 3, 2)
            self.conv2 = SigmaConv(args, 32, 64, 3, 3, 2)
            self.view1 = SigmaView((64, 8, 8), 64 * 8 * 8)
            self.fc1 = SigmaLinear(args, 64 * 8 * 8, 128)
            self.fc2 = SigmaLinear(args, 128, 10)
            
        elif args.dataset == "MNIST":
            if args.architecture == "lenet":
                self.LN1 = torch.nn.LayerNorm((1,28,28), elementwise_affine=False) 
                self.conv1 = SigmaConv(args, 1, 32, 5, 2, 3)
                self.conv2 = SigmaConv(args, 32, 64, 5, 2, 3)
                self.view1 = SigmaView((64, 4, 4), 64 * 4 * 4)
                self.fc1 = SigmaLinear(args, 64 * 4 * 4, 512)
                self.fc2 = SigmaLinear(args, 512, 10)
                
        self.forward_params = list()
        self.backward_params = list()
        for layer in [self.conv1, self.conv2, self.fc1, self.fc2]:
            forward_params, backward_params = layer.get_parameters()
            self.forward_params += forward_params
            self.backward_params += backward_params

    def forward(self, x, detach_grad=False, return_activations=True):   
        # x = self.LN1(x)
        a1 = self.conv1(x, detach_grad)
        a2 = self.conv2(a1, detach_grad)
        a2 = self.view1(a2, detach_grad)
        a3 = self.fc1(a2, detach_grad)
        a4 = self.fc2(a3, detach_grad)
        return [a1, a2, a3, a4]
        
    def reverse(self, target, detach_grad=True, return_activations=True):
        if target.shape == torch.Size([10]): 
            target = F.one_hot(target, num_classes=10).float().to(target.device)
        b3 = self.fc2.reverse(target, detach_grad)
        b2 = self.fc1.reverse(b3, detach_grad)
        b2 = self.view1.reverse(b2, detach_grad)
        b1 = self.conv2.reverse(b2, detach_grad)
        return [b1, b2, b3, target]


class SigmaLoss(nn.Module):
    def __init__(self, args):
        super(SigmaLoss, self).__init__()
        self.args = args
        self.final_criteria = nn.CrossEntropyLoss()
        self.local_criteria = compute_SCL_loss
        self.method = args.method
        
    def forward(self, activations, signals, target, method="final"):
        if method == "local":
            loss = list()
            for act, sig in zip(activations[:-1], signals[:-1]):
                loss += [self.local_criteria(act, sig, target)]
            loss += [self.final_criteria(activations[-1], target)]
            return sum(loss), loss[-1].item()
        elif method == "final":
            loss = self.final_criteria(activations[-1], target) * 1.1 - 0.1
            return loss, loss.item()
        
model = SigmaModel_SimpleCNN(args)
model.to(device)
if args.optimizer == "SGD": 
    forward_optimizer = optim.SGD(model.forward_params, lr=args.lr)
    backward_optimizer = optim.SGD(model.forward_params, lr=args.lr)
elif args.optimizer == "RMSprop": forward_optimizer = optim.RMSprop(model.forward_params, lr=args.lr, weight_decay=0)
elif args.optimizer == "Adam": forward_optimizer = optim.Adam(model.forward_params, lr=args.lr)
criteria = SigmaLoss(args)
    
with torch.no_grad():
    signals = model.reverse(torch.Tensor([0,1,2,3,4,5,6,7,8,9]).long().to(device), return_activations=True)
    
best_val_loss = float('inf')

print("No LN1")
for epoch in tqdm(range(args.epochs)):
    
    train_loss, train_counter = 0, 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx > 100: continue
        if args.method == "SIGMA":
            activations = model(data.to(device), detach_grad=True)
            signals = model.reverse(torch.Tensor([0,1,2,3,4,5,6,7,8,9]).long().to(device), detach_grad=True)
            loss, loss_item = criteria(activations, signals, target.to(device), method="local")
        elif args.method == "BP":
            activations = model(data.to(device), detach_grad=False)
            loss, loss_item = criteria(activations, signals, target.to(device), method="final")
        forward_optimizer.zero_grad(), backward_optimizer.zero_grad(), loss.backward()
        gradient_centralization(model), forward_optimizer.step(), backward_optimizer.step()
        train_loss += loss_item * len(data)
        train_counter += len(data)

    wandb.log({'train_loss': train_loss / train_counter}, step=epoch)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx > 100: continue
        activations = model(data.to(device), detach_grad=True)
        loss, loss_item = criteria(activations, signals, target.to(device), method="final")
        forward_optimizer.zero_grad(), loss.backward(), forward_optimizer.step()

    # Validation
    val_correct, val_loss, val_counter = 0, 0, 0
    with torch.no_grad():
        for data, target in val_loader:
            val_counter += len(data)
            if args.method == "SIGMA":
                activations = model(data.to(device), detach_grad=True)
                _, loss_item = criteria(activations, signals, target.to(device), method="local")
                
            elif args.method == "BP":
                activations = model(data.to(device), detach_grad=True)
                _, loss_item = criteria(activations, signals, target.to(device), method="final")
            prediction = activations[-1].detach()
            _, predicted = torch.max(prediction, 1)
            val_correct += (predicted == target.to(device)).sum().item()
            val_loss += loss_item * len(data)

    wandb.log({'val_loss': val_loss / val_counter, 
               'val_acc': val_correct / val_counter, 
               }, step=epoch)
    
    print(f"""Epoch {epoch} | train loss {train_loss / train_counter:.4f} | val loss {val_loss / val_counter:.4f} | val acc {100 * val_correct / val_counter:.4f}""")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)
        torch.save(best_model.state_dict(), f'./saved_models/{run_name}-{time_stamp}.pt')
        

# Eval on Test Set by loading the best model 
model.load_state_dict(torch.load(f'./saved_models/{run_name}-{time_stamp}.pt'))
model.eval()
correct, total = 0, 0
test_loss, test_counter = 0, 0
with torch.no_grad():
    for data, target in test_loader:
        outputs = model(data.to(device), detach_grad=False)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == target.to(device)).sum().item()
        loss, loss_item = criteria(activations, signals, target.to(device), method="final")
        test_loss += loss_item * len(data)
        test_counter += len(data)

wandb.log({'test_loss': test_loss / test_counter,
ｃｖ
print(f'Epoch: {epoch}, Test Accuracy: {100 * correct / test_counter:.4f}%')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run name: CIFAR10_SIGMA_conv32-actelu_SGD_1_2023
Files already downloaded and verified
Files already downloaded and verified
No LN1


  1%|          | 1/100 [00:13<21:53, 13.27s/it]

Epoch 0 | train loss 2.2178 | val loss 2.0178 | val acc 30.6200


  2%|▏         | 2/100 [00:27<22:11, 13.59s/it]

...

Epoch 74 | train loss 0.9998 | val loss 1.1078 | val acc 62.6200


 76%|███████▌  | 76/100 [16:55<05:23, 13.46s/it]

Epoch 75 | train loss 0.9779 | val loss 1.1026 | val acc 63.0200


 76%|███████▌  | 76/100 [17:07<05:24, 13.52s/it]


KeyboardInterrupt: 

In [ ]:
for a, s in zip(activations, signals):
    print(a.shape, s.shape)

torch.Size([64, 32, 16, 16]) torch.Size([10, 32, 32, 32])
torch.Size([64, 4096]) torch.Size([10, 64, 8, 8])
torch.Size([64, 128]) torch.Size([10, 128])
torch.Size([64, 10]) torch.Size([10, 10])


In [ ]:
Epoch 0 | train loss 2.1706 | val loss 1.7251 | val acc 38.4000
Epoch 1 | train loss 1.9300 | val loss 1.5896 | val acc 44.4500
Epoch 2 | train loss 1.8217 | val loss 1.4883 | val acc 47.6600
Epoch 3 | train loss 1.7205 | val loss 1.4662 | val acc 48.2700
Epoch 4 | train loss 1.6620 | val loss 1.3580 | val acc 52.1400

Epoch 5 | train loss 1.5822 | val loss 1.3429 | val acc 53.0200
Epoch 6 | train loss 1.5124 | val loss 1.3322 | val acc 52.9600
Epoch 7 | train loss 1.4301 | val loss 1.2645 | val acc 55.5300
Epoch 8 | train loss 1.4169 | val loss 1.2650 | val acc 55.7300
Epoch 9 | train loss 1.3360 | val loss 1.2396 | val acc 56.5400

Epoch 10 | train loss 1.3369 | val loss 1.2339 | val acc 57.2100
Epoch 11 | train loss 1.3077 | val loss 1.2344 | val acc 57.2000
Epoch 12 | train loss 1.2832 | val loss 1.2064 | val acc 58.1100
Epoch 13 | train loss 1.2695 | val loss 1.2016 | val acc 58.7200
Epoch 14 | train loss 1.2429 | val loss 1.1799 | val acc 59.7700

Epoch 15 | train loss 1.2381 | val loss 1.1819 | val acc 59.5900
Epoch 16 | train loss 1.2267 | val loss 1.1791 | val acc 59.5400
Epoch 17 | train loss 1.2239 | val loss 1.1724 | val acc 59.5000
Epoch 18 | train loss 1.1911 | val loss 1.1605 | val acc 60.5000
Epoch 19 | train loss 1.1702 | val loss 1.1587 | val acc 60.6300
Epoch 20 | train loss 1.1602 | val loss 1.1574 | val acc 60.8900

In [ ]:
Epoch 0 | train loss 2.1313 | val loss 1.7911 | val acc 35.1500
  2%|▏         | 2/100 [00:26<21:38, 13.25s/it]
Epoch 1 | train loss 1.6918 | val loss 1.5604 | val acc 44.5300
  3%|▎         | 3/100 [00:39<21:30, 13.30s/it]
Epoch 2 | train loss 1.5431 | val loss 1.4494 | val acc 48.9800
  4%|▍         | 4/100 [00:52<21:11, 13.24s/it]
Epoch 3 | train loss 1.4475 | val loss 1.3936 | val acc 50.8200
  5%|▌         | 5/100 [01:06<20:56, 13.22s/it]
Epoch 4 | train loss 1.4169 | val loss 1.3353 | val acc 53.3300
  6%|▌         | 6/100 [01:19<20:36, 13.15s/it]
Epoch 5 | train loss 1.3502 | val loss 1.2823 | val acc 55.4500
  7%|▋         | 7/100 [01:32<20:28, 13.21s/it]
Epoch 6 | train loss 1.3133 | val loss 1.2575 | val acc 57.4800
  8%|▊         | 8/100 [01:45<20:23, 13.30s/it]
Epoch 7 | train loss 1.2808 | val loss 1.2338 | val acc 58.1700
  9%|▉         | 9/100 [01:59<20:06, 13.26s/it]
Epoch 8 | train loss 1.2644 | val loss 1.2085 | val acc 59.1500
 10%|█         | 10/100 [02:12<19:51, 13.24s/it]
Epoch 9 | train loss 1.2249 | val loss 1.2015 | val acc 60.5500
 11%|█         | 11/100 [02:25<19:33, 13.19s/it]
Epoch 10 | train loss 1.2096 | val loss 1.1854 | val acc 61.0400
 12%|█▏        | 12/100 [02:38<19:22, 13.21s/it]
Epoch 11 | train loss 1.2114 | val loss 1.1818 | val acc 61.3400
 13%|█▎        | 13/100 [02:51<19:07, 13.19s/it]
Epoch 12 | train loss 1.1965 | val loss 1.1885 | val acc 62.0300
 14%|█▍        | 14/100 [03:04<18:47, 13.11s/it]
Epoch 13 | train loss 1.1937 | val loss 1.1811 | val acc 61.8700
 15%|█▌        | 15/100 [03:18<18:41, 13.20s/it]
Epoch 14 | train loss 1.2066 | val loss 1.1872 | val acc 62.3500
 16%|█▌        | 16/100 [03:31<18:22, 13.12s/it]
Epoch 15 | train loss 1.1988 | val loss 1.1774 | val acc 62.9200
 17%|█▋        | 17/100 [03:44<18:10, 13.14s/it]
Epoch 16 | train loss 1.1958 | val loss 1.1849 | val acc 61.6400
 18%|█▊        | 18/100 [03:57<17:55, 13.12s/it]
Epoch 17 | train loss 1.2015 | val loss 1.1870 | val acc 63.2400
 19%|█▉        | 19/100 [04:10<17:37, 13.06s/it]
Epoch 18 | train loss 1.1976 | val loss 1.1970 | val acc 63.2700
 20%|██        | 20/100 [04:23<17:20, 13.01s/it]
Epoch 19 | train loss 1.1864 | val loss 1.1861 | val acc 63.4800
 21%|██        | 21/100 [04:36<17:05, 12.98s/it]
Epoch 20 | train loss 1.1922 | val loss 1.2039 | val acc 62.8200